In [277]:
import numpy as np
import pandas as pd
import os

# Function to read the Dataset file 
def folder(pathname):
    if pathname.endswith('features.csv'):
        path = os.getcwd()
        file = path + pathname
        df = pd.read_csv(file, delimiter=",",header=None,index_col=False)
        return df
    else:
        path = os.getcwd()
        file = path + pathname
        df = pd.read_csv(file, delimiter=",",header=None,index_col=False)
        return df

In [278]:
## Train Dataset
file = input("Enter the File Name: ")
df_train = folder(str('/'+ file))
df_numpy_train = df_train.to_numpy()

Enter the File Name: train_features.csv


In [279]:
## Train Label
file = input("Enter the File Name: ")
df_trainLabel = folder(str('/'+ file))
df_trainLabel = df_trainLabel.drop(0,axis=0)
df_numpy_trainLabel = df_trainLabel.to_numpy()

Enter the File Name: train_label.csv


In [280]:
X = df_train.drop(0,axis=1)

In [281]:
y = df_trainLabel.drop(0,axis=1)

## KNN

In [707]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
import warnings
from sklearn.exceptions import DataConversionWarning,ConvergenceWarning
warnings.filterwarnings(action='ignore', category=DataConversionWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings(action='ignore',category=ConvergenceWarning)

# KNN Model
knn = KNeighborsClassifier()
params_knn = {'n_neighbors':np.arange(1,50)}
knn_gs = GridSearchCV(knn,params_knn,cv=10)
knn_gs.fit(X,y)

# Save Model
knn_best = knn_gs.best_estimator_
print(knn_gs.best_params_)

{'n_neighbors': 24}


## RF

In [688]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()

params_rf = {'n_estimators':[50,100,200,300,400,500],'n_jobs':[10]}
rf_gs = GridSearchCV(rf,params_rf,cv=10)
rf_gs.fit(X,y)

# Save Model
rf_best = rf_gs.best_estimator_

## LR

In [701]:
from sklearn.linear_model import LogisticRegression
import warnings
from sklearn.exceptions import DataConversionWarning
warnings.filterwarnings(action='ignore', category=DataConversionWarning)

log_reg = LogisticRegression()
params_log = {'penalty':['l1','l2']}
log_gs = GridSearchCV(log_reg,params_log,cv=5)
log_gs.fit(X,y)
log_gs.best_estimator_

/usr/local/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWar

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l1',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

## SVM

In [690]:
from sklearn.svm import SVC
svm = SVC()
params_svm = {'C':[0.001,0.01,0.1,1,10],'gamma':[0.001,0.01,0.1,1,0.005],'kernel':['rbf']}
svm_gs = GridSearchCV(svm,params_svm,cv=10)
svm_gs.fit(X,y)
svm_best = svm_gs.best_estimator_

## Stochastic Gradient Boosting

In [691]:
from sklearn.ensemble import GradientBoostingClassifier

sgb = GradientBoostingClassifier()
params_sgb = {'n_estimators':[10,100],'random_state':[5,6,7]}
sgb_gs = GridSearchCV(sgb,params_sgb,cv=10)
sgb_gs.fit(X,y)
sgb_best = sgb_gs.best_estimator_

## Adaboost

In [502]:
from sklearn.ensemble import AdaBoostClassifier
ada = AdaBoostClassifier()
params_ada = {'n_estimators':[10,100],'random_state':[5,6,7]}
ada_gs = GridSearchCV(ada,params_ada,cv=10)
ada_gs.fit(X,y)
ada_best = ada_gs.best_estimator_

## Bagging

In [692]:
from sklearn.ensemble import BaggingClassifier

bagging_clf_svm = BaggingClassifier(svm_best)
bagging_clf_rf = BaggingClassifier(rf_best)
bagging_clf_log = BaggingClassifier(log_reg)
bagging_clf_knn = BaggingClassifier(knn_best)

## Prepare Train & Test Split for Accuracy

In [693]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.33)

## Voting Classifier

In [750]:
### from sklearn.ensemble import VotingClassifier

estimators = [('svm',svm_best),('rf',rf_best),('log_reg',log_gs),('ada',ada_best)] 

# [('rf',rf_best),('log_reg',log_reg),('ada',ada_best),('sgb',sgb_best)] - kernel, gamma
# ('rf',rf_best),('log_reg',log_reg),('ada',ada_best),('sgb',sgb_best)
# ('knn',knn_best),('svm',svm_best),('rf',rf_best),('log_reg',log_reg),('sgb',sgb_best), ('ada',ada_best)
ensemble = VotingClassifier(estimators, voting='hard')
ensemble.fit(X_train,y_train)
#y_pred = ensemble.predict(X_test)
ensemble.score(X_test,y_test)


0.928

## Test

In [738]:
## Train Dataset
file = input("Enter the File Name: ")
df_test = folder(str('/'+ file))


Enter the File Name: test_features.csv


In [739]:
X = df_test.iloc[:,1:]
ids = df_test.iloc[:,0]
ids = ids.rename(columns={0:'id'})

In [740]:
y = ensemble.predict(X)

In [741]:
y_df = pd.DataFrame(y)
y_df = y_df.rename(columns={0:'label'})

In [742]:
final_y = pd.concat([ids,y_df],axis=1)
final_yl = final_y.rename(columns={0:'id'})

In [743]:
final_yl.to_csv('submission8.csv',index=False)

## Results

Highest Accuracy using Training Dataset = Fluctuating between 0.91 - 0.93
Classifiers used are SVM, Random Forest, Log_regression, Ada_Boost

Best Parameters Observed: 

// 1. SVM
C = 0.001
Gamma = 0.001
Kernel = rbf

// 2. Random Forest
n_estimators = 50
n_jobs = 10

// 3. Log Regression
Cross Validation = 5
penalty = l2

// 4. AdaBoost
n-estimators = 100
random_state = 5

In [748]:
ensemble.named_estimators

{'svm': SVC(C=0.001, cache_size=200, class_weight=None, coef0=0.0,
     decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
     max_iter=-1, probability=False, random_state=None, shrinking=True,
     tol=0.001, verbose=False),
 'rf': RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                        max_depth=None, max_features='auto', max_leaf_nodes=None,
                        min_impurity_decrease=0.0, min_impurity_split=None,
                        min_samples_leaf=1, min_samples_split=2,
                        min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=10,
                        oob_score=False, random_state=None, verbose=0,
                        warm_start=False),
 'log_reg': GridSearchCV(cv=5, error_score='raise-deprecating',
              estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                           fit_intercept=True,
                                           int